In [1]:
# Run the below line in terminal to install the required packages
# pip install -r ./resources/requirements.txt


In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster

In [ ]:
# Load the CSV file with low_memory=False to address DtypeWarning
csv_path = "https://raw.githubusercontent.com/kkruel8100/Crime-Analysis/main/resources/Crime.csv"
df = pd.read_csv(csv_path, low_memory=False)

In [ ]:
# Convert Latitude and Longitude to numeric, forcing errors to NaN to ensure we can filter them out
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

In [ ]:
# Filter out rows with NaN or 0.0 for Latitude or Longitude
df = df.dropna(subset=['Latitude', 'Longitude'])
df = df[(df['Latitude'] != 0) & (df['Longitude'] != 0)]

# Categorize crimes based on 'Crime Name1'
df['Crime Against Person'] = df['Crime Name1'].apply(lambda x: 'Yes' if 'Crime Against Person' in str(x) else 'No')

In [ ]:
# Convert the cleaned DataFrame to a GeoDataFrame for spatial plotting
gdf_crime = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs='EPSG:4326')

In [ ]:
# Specify the latitude and longitude bounds for Maryland
md_latitude = 38.80785
md_longitude = -76.84997
md_zoom_start = 8

In [ ]:
# Create the map centered around Maryland
map_md = folium.Map(location=[md_latitude, md_longitude], zoom_start=md_zoom_start)

In [ ]:
# Create marker clusters for Crimes Against Person (red) and Other Crimes (blue)
marker_cluster_red = MarkerCluster(name='Crimes Against Person', show=False).add_to(map_md)
marker_cluster_blue = MarkerCluster(name='Other Crimes', show=False).add_to(map_md)

In [ ]:
# Add points to the map
for idx, row in gdf_crime.iterrows():
    if row['Crime Against Person'] == 'Yes':
        folium.CircleMarker(location=[row['Latitude'], row['Longitude']],
                            radius=5,
                            color='red',
                            fill=True,
                            fill_color='red').add_to(marker_cluster_red)
    else:
        folium.CircleMarker(location=[row['Latitude'], row['Longitude']],
                            radius=5,
                            color='blue',
                            fill=True,
                            fill_color='blue').add_to(marker_cluster_blue)

In [ ]:
# Add LayerControl to allow toggling between layers
folium.LayerControl(collapsed=False).add_to(map_md)

In [ ]:
# Display the map in a Jupyter notebook
map_md